### Create DataSet

In [ ]:
from langsmith import Client
client = Client()

In [ ]:
examples = client.list_shared_examples(share_token="206cb644-3dd4-4713-878b-cd1ae980c103")

In [ ]:
new_examples = set()

In [ ]:
for example in examples:
    new_examples.add((example.inputs["messages"][0]["content"],"supervisor"))


In [ ]:
import csv


file_path = "/home/ytaghzou/Téléchargements/Not valid questions - Sheet1.csv"

# Open the CSV file
with open(file_path, newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    
    # Skip the header (if there is one)
    header = next(csvreader)
    
    # Read the rows
    for row in csvreader:
        new_examples.add((row[0].strip(),"__end__"))


In [ ]:
len(new_examples)

In [ ]:
new_examples = list(new_examples)

In [ ]:
dataset_name = "KGBot Evaluation: Validation Step"
if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(dataset_name=dataset_name)
client.create_examples(
    inputs = [{"question":ex[0]} for ex in new_examples],
    outputs = [{"route": ex[1]} for ex in new_examples],
    dataset_id=dataset.id
)

# Prepare Evaluation

In [ ]:
from langsmith import Client
client = Client()

In [ ]:
dataset = client.read_dataset(dataset_name="KGBot Evaluation: Validation Step")
dataset.example_count

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..', '..'))  # Adjust the dots as needed
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

if os.path.exists("./app/config/logs") == False:
    os.makedirs("./app/config/logs")

In [ ]:
from app.core.main_studio import app
from langchain_core.messages import HumanMessage, AIMessage
from app.core.workflow.langraph_workflow import router


# Target function for running the relevant step
async def run_validator_classifier(input: str) -> dict:

    state = {
        "messages": [
            HumanMessage(f"Starting the processing of the question: {input['question']}", name="Entry_Agent")
        ]
    }
    answer = await app.nodes['Validator'].ainvoke(state)
    goto = router(answer)
    return {"route": goto}

In [ ]:
# Evaluator
def correct(outputs: dict, reference_outputs: dict) -> bool:
    """Check if the agent chose the correct route."""
    return outputs["route"] == reference_outputs["route"]

In [ ]:
# Run evaluation
experiment_results = await client.aevaluate(
    run_validator_classifier,
    data=dataset.name,
    evaluators=[correct],
    experiment_prefix="kgbot-evaluation-single-step-deepseek_r1",
    max_concurrency=4,
)
experiment_results.to_pandas()